In [ ]:
import requests
from bs4 import BeautifulSoup
from astropy.io import fits
import matplotlib.pyplot as plt
from desispec.io.spectra import read_spectra
from desispec.coaddition import coadd_cameras
import numpy as np
from desitarget.targetmask import desi_mask
from astropy.table import Table
from desispec.resolution import Resolution
from scipy.optimize import curve_fit

In [ ]:
### Generates Lists of all the links needed to download the data
r_i = requests.get("https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/healpix/main/dark/") 
soup_i = BeautifulSoup(r_i.content, "html.parser")
all_links_i = soup_i.find_all("a")
linkList_i = [link['href'] for link in all_links_i[1:]] # creates a list of all the first indicies of the coaad files

all_lists = []
for i in range(len(linkList_i)):
    r_j = requests.get("https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/healpix/main/dark/" + str(linkList_i[i]))
    soup_j = BeautifulSoup(r_j.content, "html.parser")
    all_links_j = soup_j.find_all("a")
    linkList_j = [link['href'] for link in all_links_j] 
    all_lists.append(linkList_j[1:]) # creates a list of all the second indicies of the coaad files for each of the first indicies
    

In [ ]:
### Initializing
tile = set()
zArray = []
RaArray = []
DecArray = []
tidArray = []
hdul = fits.open("qso_cat_dr1_main_dark_healpix_zlya-v0.fits") # lya quasar catalog
data = hdul[1].data

def gauss(x,sig,A,mu): # gaussian function
    return A*np.exp(-((x-mu)**2) / (2*sig**2))

In [ ]:
### Dowload and extract data
for i in range(1): #range(len(linkList_i)):
    for j in range(1): #range(len(all_lists[i])):
        if int(all_lists[i][j].replace("/","")) in tile: # checks if tile has already been downloaded and all spectra extracted
            print('coadd-main-dark-'+  str(all_lists[i][j]).replace("/","") + '.fits' + ' has already been downloaded')
            continue
        else:
            downFile = ('https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/healpix/main/dark/' +  str(linkList_i[i]) # uses url generated from linkList and all_lists
                        + str(all_lists[i][j]) + 'coadd-main-dark-'+  str(all_lists[i][j]).replace("/","") + '.fits')
            !wget "{downFile}"
            print(all_lists[i][j].replace("/",""))

            spec = read_spectra("coadd-main-dark-" + all_lists[i][j].replace("/","") + ".fits" ) # gets spectra data from the coaad file
            fibermap = spec.fibermap
            coadd_dict = {tid: ii for ii, tid in enumerate(fibermap["TARGETID"])} # creates a dictionary of indicies with the target id as the key

            for ii, tid in enumerate(data["TARGETID"]):
                if (data["Z"][ii] < 5.4) and (data["ZWARN"][ii] == 0) and (data["BI_CIV"][ii] == 0): #checks that Z < Z_max, ZWARN = 0, and BI_CIV < BI_CIV_max
                    if tid in coadd_dict: # checks that the target id from the lya QSO catelog is in the coadd file
                        tidArray.append(str(tid)) #generates a list of target id's of valid QSOs -> saved as str because of floating point problems
                        zArray.append(data["Z"][ii]) # generates a list of redshift of all the valid QSOs
                        RaArray.append(data["TARGET_RA"][ii]) # generates a list of RA of all the valid QSOs
                        DecArray.append(data["TARGET_DEC"][ii]) # generates a list of DEC of all the valid QSOs
                        spec_one = spec[coadd_dict[tid]]
                        wave = coadd_cameras(spec_one).wave['brz'] # extracts and combines the three arms for wave, flux, and ivar
                        flux = coadd_cameras(spec_one).flux['brz'][0]
                        ivar = coadd_cameras(spec_one).ivar['brz'][0]

                        pixel_mask = coadd_cameras(spec_one).mask['brz'][0] # saves array of masked/unmasked pixels
                        res = coadd_cameras(spec_one).resolution_data['brz'][0] # gets resolution data from current spectra
                        R  = Resolution(res) # creates sparse resolution matrix
                        sigArray = []
                        rArray = []
                        for jj in range(len(wave)): # generates
                            kernel = res[:, jj]  # shape: (ndiag,), centered at wave[i]
                            # Compute the pixel offsets relative to center
                            offsets = R.offsets
                            kernel_wave = wave[jj] + (offsets * np.gradient(wave)[i])
                            if np.nan in kernel:
                                sigArray.append(np.nan)
                            elif np.inf in kernel:
                                sigArray.append(np.inf)
                            else:
                                # Get the resolution kernel (ISF) at pixel i
                                kernel = res[:, jj]  # shape: (ndiag,), centered at wave[i]

                                # Compute the pixel offsets relative to center
                                offsets = R.offsets
                                kernel_wave = wave[jj] + (offsets * np.gradient(wave)[i])
                                peak = max(kernel)

                                parameters = curve_fit(gauss, kernel_wave, kernel, p0=[0.5, 1, kernel_wave[5]])
                                sig_fit, A_fit, mu_fit = parameters[0]
                                sigArray.append(sig_fit)
                                print(sig_fit)
                        t = Table()
                        t['WAVE'] = wave # formats the data into a table
                        t['FLUX'] = flux
                        t['IVAR'] = ivar
                        t['PIXEL_MASK'] = pixel_mask 
                        t['SIGMA_PIXEL'] = sigArray
                        t.write(str(tid) + '.fits',"overwrite=True") # writes the data to a fits file for analysis

            tile.add(int(all_lists[i][j].replace("/",""))) # used to make sure that if code stops while downloading, the last downloaded tile is saved so we dont have to restart
            filename = "coadd-main-dark-" + all_lists[i][j].replace("/","") + ".fits"
            !rm -rf "{filename}" # removes the downloaded coadd file
        
t2 = Table()
t2['TARGETID'] = tidArray 
t2['Z'] = zArray
t2['TARGET_RA'] = RaArray
t2['TARGET_DEC'] = DecArray
t2.write('QSO_catalog.fits',"overwrite=True") # writes a file with tid and redshifts of all the valid QSOs
